# Train classifiers based on paper
* 

In [2]:
import os
import sys
import glob
import shutil
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.data
from torchvision.models import resnet18, resnet34, resnet50, resnet101, vgg16, inception, inception_v3
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

# import my functions:
from tile_train_functions import*

# Set it to use GPU1
torch.cuda.set_device(0)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/home/rene/code/camelyon/src
True
0


## Inception
(Deep Learning for Identifying Metastatic Breast Cancer arXiv:1606.05718v1)

In [2]:
PATH = '/media/rene/Data/camelyon_out/tiles_224_100t'
num_workers = 4
batch_size=64

sz=224
dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid')

epochs = 12
save_path = '/media/rene/Data/camelyon_out/tiles_224_100t/sample/models'

###############################



import torch
import pretrainedmodels.utils as utils

load_img = utils.LoadImage()

# transformations depending on the model
# rescale, center crop, normalize, and others (ex: ToBGR, ToRange255)
tf_img = utils.TransformImage(model) 

path_img = 'data/cat.jpg'

input_img = load_img(path_img)
input_tensor = tf_img(input_img)         # 3x400x225 -> 3x299x299 size may differ
input_tensor = input_tensor.unsqueeze(0) # 3x299x299 -> 1x3x299x299
input = torch.autograd.Variable(input_tensor,
    requires_grad=False)

output_logits = model(input) # 1x1000



##################################

# ????????
model_arch = inception
model_name = 'inception'

model_ft = model_arch(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

best_acc, model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                   num_epochs=epochs)
torch.save(model_ft.state_dict(), os.path.join(save_path, model_name+'_full_dataset'))

TypeError: 'module' object is not callable

In [ ]:
dataloaders_valid, dataset_sizes_valid = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid')
dataloaders_test, dataset_sizes_test = make_batch_gen(PATH, batch_size, num_workers, valid_name='test')
criterion = nn.CrossEntropyLoss()

# get the proper model architecture
model_ft = model_arch(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

# load the saved weights
model_ft.load_state_dict(torch.load(os.path.join(save_path, 'inception_full_dataset')))
print('Validation: ', model_name[idx])
valid_loss, valid_acc = eval_model(model_ft, dataloaders_valid['valid'], dataset_sizes_valid['valid'], criterion)

print('Test: ', model_name[idx])
test_loss, test_acc = eval_model(model_ft, dataloaders_test['test'], dataset_sizes_test['test'], criterion)
results[model_name[idx]] = [valid_acc, test_acc]

## Inception v3
(Detecting Cancer Metastases on Gigapixel Pathology Images)

In [2]:
PATH = '/media/rene/Data/camelyon_out/tiles_224_100t'
num_workers = 4
batch_size=32

dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid', size=299)

epochs = 12
save_path = '/media/rene/Data/camelyon_out/tiles_224_100t/sample/models'

# ????????
model_arch = inception_v3
model_name = 'inception_v3'

model_ft = model_arch(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

best_acc, model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, epochs,
                                dataloaders, dataset_sizes)
torch.save(model_ft.state_dict(), os.path.join(save_path, model_name+'_full_dataset'))

FileNotFoundError: [Errno 2] No such file or directory: '/media/rene/Data/camelyon_out/tiles_224_100t/train'

In [2]:
PATH = '/media/rene/Data/data/camelyon_out/tiles_224_100t'
save_path = '/media/rene/Data/data/camelyon_out/tiles_224_100t/sample/models'
num_workers = 4
batch_size=32

dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, 
                                            valid_name='valid', test_name ='test', size=299)
criterion = nn.CrossEntropyLoss()

model_arch = inception_v3
model_name = 'inception_v3'

# get the proper model architecture
model_ft = model_arch(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

In [9]:
# load the saved weights
model_ft.load_state_dict(torch.load(os.path.join(save_path, 'inception_v3_full_dataset')))
print('Validation: ', model_name)
valid_loss, valid_acc = eval_model(model_ft, dataloaders_valid['valid'], dataset_sizes_valid['valid'], criterion)

print('Test: ', model_name)
test_loss, test_acc = eval_model(model_ft, dataloaders_test['test'], dataset_sizes_test['test'], criterion)

Validation:  inception_v3
Loss: 0.1675 Acc: 0.9317
Test:  inception_v3
Loss: 0.3418 Acc: 0.8531


NameError: name 'results' is not defined

## Get sensitivity/specificity

In [3]:
def get_metrics(all_labels, all_preds):
    """https://stackoverflow.com/questions/31324218"""
    metrics = {}
    cm = confusion_matrix(all_labels, all_preds)
    FP = cm.sum(axis=0) - np.diag(cm)
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)

    # Sensitivity, hit rate, recall, or true positive rate
    metrics['TPR'] = TP/(TP+FN)
    # Specificity or true negative rate
    metrics['TNR'] = TN/(TN+FP) 
    # Precision or positive predictive value
    metrics['PPV'] = TP/(TP+FP)
    # Negative predictive value
    metrics['NPV'] = TN/(TN+FN)
    # Fall out or false positive rate
    metrics['FPR'] = FP/(FP+TN)
    # False negative rate
    metrics['FNR'] = FN/(TP+FN)
    # False discovery rate
    metrics['FDR'] = FP/(TP+FP)
    # Overall accuracy
    metrics['ACC'] = (TP+TN)/(TP+FP+FN+TN)
    return metrics

def get_metrics_bin(all_labels, all_preds):
    metrics = {}
    cm = confusion_matrix(all_labels, all_preds)
    TP = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TN = cm[1][1]
    # Sensitivity, hit rate, recall, or true positive rate
    metrics['TPR'] = TP/(TP+FN)
    # Specificity or true negative rate
    metrics['TNR'] = TN/(TN+FP) 
    # Precision or positive predictive value
    metrics['PPV'] = TP/(TP+FP)
    # Negative predictive value
    metrics['NPV'] = TN/(TN+FN)
    # Fall out or false positive rate
    metrics['FPR'] = FP/(FP+TN)
    # False negative rate
    metrics['FNR'] = FN/(TP+FN)
    # False discovery rate
    metrics['FDR'] = FP/(TP+FP)
    # Overall accuracy
    metrics['ACC'] = (TP+TN)/(TP+FP+FN+TN)
    return metrics

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

model_ft.load_state_dict(torch.load(os.path.join(save_path, 'inception_v3_full_dataset')))

all_labels, all_preds = get_preds(model_ft, dataloaders['valid'], dataset_sizes['valid'], criterion)
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Validation sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

all_labels, all_preds = get_preds(model_ft, dataloaders['test'], dataset_sizes['test'], criterion)
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Test sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

<class 'list'>
Validation sensitivity: 0.9317942107113559, specificity: 0.9315538141300355, Acc: 0.9316766467065868
<class 'list'>
Test sensitivity: 0.759115152574976, specificity: 0.9588501673557787, Acc: 0.8530678397777264


## ResNet

In [3]:
PATH = '/home/rene/data/tiles_224_100t'
save_path = '/home/rene/data/tiles_224_100t/models'
num_workers = 4
batch_size=16

dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, 
                                            valid_name='valid', test_name ='test', size=224)

model_arch = resnet34
model_name = 'resnet34_scratch_no_samp_5'

# get the proper model architecture
model_ft = model_arch(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name)))

In [4]:
all_labels, all_preds = get_preds(model_ft, dataloaders['valid'], dataset_sizes['valid'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Validation sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

all_labels, all_preds = get_preds(model_ft, dataloaders['test'], dataset_sizes['test'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Test sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

Validation sensitivity: 0.9131319986426875, specificity: 0.9443949611909912, Acc: 0.9278443113772455
Test sensitivity: 0.7306666666666667, specificity: 0.9653986451276707, Acc: 0.8349617967122019
